# Bioinformatic data minign
The goal of this activity is to put into practice the bioinformatics dataa mining skills adquired so far, such as using APIs to access biological and chemical databases and leveraging specielized libraries for data handling.

In [2]:
# Importing necessary library
import requests
import pandas as pd

### Retrieval information about proteins and chemical compounds

I implemented a function that takes a list of Protein Data Bank (PDB) IDs as input and downloads the corresponding `.cif` files from the PDB database.

In [5]:
# Base URL for PDB file requests
base_url = "https://www.rcsb.org/pdb/files/"

def cif_downloader(id_list, url):
    """
    Downloads CIF files from the RCSB PDB for a given list of PDB IDs.
    
    Args:
        id_list: A list of PDB IDs (strings)
        url: A base url for file requests

    Returns:
        None
    """
        
    for id in id_list:
        # Construct the URL for the specific PDB ID
        response = requests.get(f"{base_url}{id}.cif")
        
        # Check if the request was succesful
        if response.status_code == 200:
            # Save the CIF file in chuncks
            with open(f"{id}.cif", "wb") as cif_file:
                for chunk in response.iter_content(chunk_size=1024):
                    cif_file.write(chunk)
            print(f"CIF file for {id} downloaded successfully.\n")
        else:
            print(f"Error ({response.status_code}) downloading CIF file for {id}.")

# Ejemplo de uso
lista_ids_ejemplo = ['1tup']
cif_downloader(lista_ids_ejemplo, base_url)

CIF file for 1tup downloaded successfully.



In this section, using the protein ID '1tup', its UniProt identifier is determined through the UniProt API. Once the identifier is obtained, the following information is extracted from UniProt and stored in a DataFrame:
1. publication date
2. last modification date
3. review status (Swiss-Prot or TrEMBL)
4. gene name and synonyms
5. organism
6. full protein name
7. single-letter amino acid sequence
8. associated PDB IDs.

In [ ]:
# Create an empty DataFrame to store the data
df_1tup = pd.DataFrame()

# Construct the URL for the UniProt API search
url_1tup = f'https://rest.uniprot.org/uniprotkb/search?query=1tup'

# Send a GET request to the UniProt API
response_1tup = requests.get(url_1tup)

# Process the response data
if response_1tup.status_code == 200:
    data_1tup = response_1tup.json()

    # Iterate over each result in the JSON response
    for result in data_1tup["results"]:
        # Handle potential missing synonyms data
        synonyms = result.get("genes", [{}])[0].get("synonyms", [])

        # Create a new row for the DataFrame
        new_row = {
            'Uniprot_id': result["uniProtkbId"],
            'Publication_date': result["entryAudit"]["firstPublicDate"],
            'Modification_date': result["entryAudit"]["lastAnnotationUpdateDate"],
            'Reviewed': ["True" if "Swiss-Prot" in result["entryType"] else "False"],
            'Gene_name': result["genes"][0]["geneName"]["value"],
            'Synonyms': [synonyms if synonyms else None],
            'Organism': result["organism"]["scientificName"],
            # Add the full protein name
            'Protein_name': result["proteinDescription"]["recommendedName"]["fullName"]["value"],
            'PDB_ids': result["primaryAccession"],
        }

        # Append the new row to the DataFrame
        df_1tup = pd.concat([df_1tup, pd.DataFrame(new_row, index=[0])], ignore_index=True)
else:
    print(f"Error {response_1tup.status_code} occurred when fetching data.")

Next, we investigate UniProt to determine if the '1tup' entry includes information about any cofactors. If a cofactor is identified, we utilize the PubChem API to extract the following data using its CID:
1. PubChem compound identifier (CID)
2. Exact molecular weight
3. InChI
4. InChIKey
5. IUPAC names

This information is then stored in a DataFrame.

In [16]:
cofactor_name = data_1tup.get("results")[0].get("comments")[1].get("cofactors")[0].get("name")

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Extract cofactor information from UniProt data
cofactor_name = data_1tup.get("results")[0].get("comments")[1].get("cofactors")[0].get("name")
chebi_id = data_1tup.get("results")[0].get("comments")[1].get("cofactors")[0].get("cofactorCrossReference").get("id")

# Construct the PubChem API URL for the cofactor (CID 32051)
url_cofactor = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/32051/JSON"

# Create an empty DataFrame to store cofactor data
df_cofactor = pd.DataFrame()

# Download cofactor information from PubChem
response_cofactor = requests.get(url_cofactor)

if response_cofactor.status_code == 200:
    data_cofactor = response_cofactor.json()

    # Extract desired information about the cofactor
    new_row = {
        "Compound": "CHEBI:29105 - zinc(2+)",  # Assuming this is static information

        "Pubchem_id": data_cofactor.get("Record").get("RecordNumber"),

        "Peso_molecular": data_cofactor.get("Record").get("Section")[2].get("Section")[0].get("Section")[0].get("Information")[0].get("Value").get("StringWithMarkup")[0].get("String"),
        # Consider using a loop or list comprehension for potentially nested structures

        "Inchi": data_cofactor.get("Record").get("Section")[1].get("Section")[1].get("Section")[1].get("Information")[0].get("Value").get("StringWithMarkup")[0].get("String"),
        # Consider using a loop or list comprehension for potentially nested structures

        "Inchikey": data_cofactor.get("Record").get("Section")[1].get("Section")[1].get("Section")[2].get("Information")[0].get("Value").get("StringWithMarkup")[0].get("String"),
        # Consider using a loop or list comprehension for potentially nested structures

        "Iupac_name": data_cofactor.get("Record").get("Section")[1].get("Section")[1].get("Section")[0].get("Information")[0].get("Value").get("StringWithMarkup")[0].get("String"),
        # Consider using a loop or list comprehension for potentially nested structures
    }

    # Add the new row to the DataFrame
    df_cofactor = pd.concat([df_cofactor, pd.DataFrame(new_row, index=[0])], ignore_index=True)

else:
    print(f"Error {response_cofactor.status_code} occurred when fetching cofactor data.")

"""Biopython y rdkit son librerías muy útiles para manipular datos de origen biológico y químico, respectivamente.
Utilízalas para completar estas tareas sobre el archivo 4ogq.cif: """


In [ ]:
# Reutilizamos la función creada en el punto 1
descargar_cifs(["4ogq"])

"""A. Parséalo con MMCIFParser() y guarda en una lista todas las heteromoléculas (sin incluir las aguas). (0,5)"""

from Bio.PDB.MMCIFParser import MMCIFParser
parser = MMCIFParser(QUIET=True)
structure = parser.get_structure('4ogq', "./4ogq.cif")

heteromoleculas = []
for modelo in structure:
    for cadena in modelo:
        for residuo in cadena:
            if residuo.get_resname() != 'HOH': #Todas las heteromoleculas distintas a agua
                heteromoleculas.append(residuo.id)

print(f"Hay {len(heteromoleculas)} heteromoleculas en la estructura (sin incluir las aguas)")


"""B. Parseálo con MMCIF2Dict() en un diccionario, extrae la información sobre la clave '_pdbx_entity_nonpoly' 
y guarda en un DataFrame la información sobre el nombre de cada una de las heteromoléculas,
así como su identificador de tres letras. (0,5)"""


In [ ]:
from Bio.PDB.MMCIFParser import MMCIF2Dict
import pandas as pd

dic_4ogq = MMCIF2Dict("./4ogq.cif")

df_4ogp = pd.DataFrame({"entitty_id" : dic_4ogq.get("_pdbx_entity_nonpoly.entity_id"),
                        "entity_name" : dic_4ogq.get("_pdbx_entity_nonpoly.name"),
                        "comp_id" : dic_4ogq.get("_pdbx_entity_nonpoly.comp_id")})


"""C. Toma el nombre de cada heteromolécula y mediante la API de Pubchem consigue su SMILES,
para los casos que sea posible.
Transforma estos SMILES con rdkit en SDF y añade ambas columnas al DataFrame del Apartado 2B. (1)"""


In [ ]:
import pubchempy as pcp
from rdkit import Chem    

for index, row in df_4ogp.iterrows():
    nombre = row["entity_name"]

    try:
        # Buscamos en pubchem por el nombre de cada compuesto
        compound = pcp.get_compounds(nombre, "name")

        if compound:
            # Tomamos el primer hallazgo de cada busqueda y buscamos los smiles
            smiles = compound[0].isomeric_smiles
            # Agregamos al df
            df_4ogp.loc[index, "smiles"] = smiles
            
            # Convertimos a sdf
            mol_smiles = Chem.MolFromSmiles(smiles)
            sdf = Chem.MolToMolBlock(mol_smiles)
            # Agregamos al df
            df_4ogp.loc[index, "SDF"] = sdf
            
    except:
        print(f"Error en: {nombre}")


"""D. Genera un archivo .sdf en el cual guardes todas las moléculas SDF.
Añade para cada una de ellas un campo que incluya información sobre su 'Molecular_weight'.
Ayúdate de rdkit para calcular dicho campo y asegúrate que el archivo que has creado
puede utilizarse para crear un objeto mol de rdkit con cada una de las moléculas. (1,5)"""
